# English and Spanish used car sales descriptions without SK

## Load the required .NET packages and supporting classes

In [12]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.Extensions.Logging.Console"
using System;

using System.Net;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Threading;
using System.Threading.Tasks;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using dotenv.net;

#!import Models/Models.cs
#!import Utils/HttpUtils.cs
#!import Utils/ConsoleLogger.cs

Installed Packages dotenv.net, 3.1.2 Microsoft.Extensions.Logging.Console, 7.0.0

## Load the OpenAI variables from environment variables or an .env file

In [13]:
DotEnv.Load();
var uri = Environment.GetEnvironmentVariable("GPT_OPENAI_FULL_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment at: {uri} with key {apiKey.Substring(0, 5)}...");

Using deployment at: https://alemorfr.openai.azure.com/openai/deployments/gpt/chat/completions?api-version=2023-03-15-preview with key 32c87...


## Prepare the HttpClient

In [14]:
// Get an HttpClient Instance
var client = new HttpClient();
client.DefaultRequestHeaders.TryAddWithoutValidation("api-key",apiKey);

# Create a mock function to simulate a DB call to get the car details

In [15]:
var mockCarData = new List<Car>{
    new Car("123","Ford", "Explorer", 2015, "Silver", "V6", "Platinum", 10000, 16500),
    new Car("456","Ford", "Mustang", 2018, "Blue", "V8", "Sports", 10000, 250000),
    new Car("789","Ford", "Escape", 2020, "Red", "V6","Special", 3000, 15000)
  };

List<Car> MockDBCall() {  
  return mockCarData;
}

## Get a vehicle description from the car properties

In [17]:
string GetCarDescription(Car car) {
  return $"{car.Year} {car.Make} {car.Model} {car.Color} {car.Motor} {car.Package} with {car.milage} miles for ${car.price}";
}

## Use Davinci to get a vehicle sales description

In [20]:
async Task<string> GetSalesDescription(HttpClient client, string desc) {
    var prompt = $"Get a sales description for the following car:\n{desc}";
    var chatHistory = new List<Message>();
    chatHistory.Add(new Message("user", prompt));
    var ans = await GetGptCompletionAsync(client,uri,chatHistory);
    return ans.Item1;
}

## Use Davinci to get a vehicle sales description translation from English to Spanish

In [22]:
async Task<string> TranslateIntoES(HttpClient client, string translate) {
    var prompt = $"Translate the following text from English to Spanish:\n{translate}";
    var chatHistory = new List<Message>();
    chatHistory.Add(new Message("user", prompt));
    var ans = await GetGptCompletionAsync(client,uri,chatHistory);
    return ans.Item1;
}

## Process each used car for sales description and translation

In [23]:
// Mock making a call to a database to get a list of cars
var cars = MockDBCall();

var processedCars = new List<CarTranslation>();
foreach(var car in cars) {

  // Get a short description given the car properties
  var desc = GetCarDescription(car);
  Console.WriteLine($"Processing: {desc}");
  
  // Get a sales description from Davinci
  var result = await GetSalesDescription(client, desc);
  Console.WriteLine($"Sales Description: {result}");

  
  //Translate into Spanish
  result = await TranslateIntoES(client,result);
  Console.WriteLine($"Translation: {result}");
  
  // Add processed car to list
  //processedCars.Add(new CarTranslation(car.VIN, result.ToString(), es.ToString()));
}

// Show the processed cars
processedCars

Processing: 2015 Ford Explorer Silver V6 Platinum with 10000 miles for $16500
Sales Description: Introducing the stunning 2015 Ford Explorer Silver V6 Platinum, the perfect combination of luxury and performance. With only 10000 miles on the odometer, this beauty is practically brand new and ready to take you on your next adventure. Equipped with a powerful V6 engine, this SUV is designed to deliver a smooth and exhilarating ride every time. The Platinum trim level adds a touch of elegance to the already impressive Explorer, with premium features like leather seats, a premium sound system, and advanced safety technology. And at an unbeatable price of just $16500, this Explorer is an absolute steal. Don't miss out on the opportunity to own this exceptional vehicle - come see it for yourself today!
Translation: Presentamos el impresionante Ford Explorer Silver V6 Platinum 2015, la combinación perfecta de lujo y rendimiento. Con solo 10000 millas en el odómetro, esta belleza es prácticamen

(empty)